In [1]:
import os
import time
import torch
import argparse

from model import SASRec
from utils import *
import csv
import pandas as pd
from tqdm import tqdm
'''
大概是这样的，这个代码要求用户和项目id从1开始，所以我前几行就做了这个操作
然后需要把我们的csv数据变成txt，或者你自己改一下他的源代码
class args类之后的所有代码是在生成提交文件
'''

'\n大概是这样的，这个代码要求用户和项目id从1开始，所以我前几行就做了这个操作\n然后需要把我们的csv数据变成txt，或者你自己改一下他的源代码\nclass args类之后的所有代码是在生成提交文件\n'

In [2]:
df = pd.read_csv('data/train_dataset.csv')
df

,user_id,item_id
0,0,257
1,0,267
2,0,5555
3,0,3637
4,0,1795
...,...,...
5869626,49801,4655
5869627,49801,5091
5869628,49801,5294
5869629,49801,5608


In [2]:
data  = pd.read_csv('./data/ml-1m.txt', header=None)

In [3]:
print(data)

             0     1
0            0   257
1            0   267
2            0  5555
3            0  3637
4            0  1795
...        ...   ...
5869626  49801  4655
5869627  49801  5091
5869628  49801  5294
5869629  49801  5608
5869630  49801  5683

[5869631 rows x 2 columns]


In [4]:
data[0] = data[0] + 1

In [5]:
print(data)

             0     1
0            1   257
1            1   267
2            1  5555
3            1  3637
4            1  1795
...        ...   ...
5869626  49802  4655
5869627  49802  5091
5869628  49802  5294
5869629  49802  5608
5869630  49802  5683

[5869631 rows x 2 columns]


In [6]:
data[1] = data[1] + 1

In [7]:
print(data)

             0     1
0            1   258
1            1   268
2            1  5556
3            1  3638
4            1  1796
...        ...   ...
5869626  49802  4656
5869627  49802  5092
5869628  49802  5295
5869629  49802  5609
5869630  49802  5684

[5869631 rows x 2 columns]


In [8]:
data.to_csv('./data/ml-1m.txt', index=0, header=0)

In [9]:
class args():
    def __init__(self):
        self.dataset = 'ml-1m'
        self.batch_size = 128
        self.lr = 0.001
        self.maxlen = 200
        self.hidden_units = 50
        self.num_blocks = 2
        self.num_epochs = 60
        self.num_heads  = 1
        self.dropout_rate = 0.3
        self.l2_emb = 0.0
        self.inference_only = False
        self.state_dict_path = None
        self.device = 'cpu'

In [10]:
args = args()

In [11]:
model = SASRec(53424, 10000, args)

In [12]:
model.load_state_dict(torch.load('./ml-1m_default/SASRec.epoch=60.lr=0.001.layer=2.head=1.hidden=50.maxlen=200.pth', map_location=torch.device(args.device)))

<All keys matched successfully>

In [13]:
model.eval()

SASRec(
  (item_emb): Embedding(10001, 50, padding_idx=0)
  (pos_emb): Embedding(200, 50)
  (emb_dropout): Dropout(p=0.3, inplace=False)
  (attention_layernorms): ModuleList(
    (0): LayerNorm((50,), eps=1e-08, elementwise_affine=True)
    (1): LayerNorm((50,), eps=1e-08, elementwise_affine=True)
  )
  (attention_layers): ModuleList(
    (0): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=50, out_features=50, bias=True)
    )
    (1): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=50, out_features=50, bias=True)
    )
  )
  (forward_layernorms): ModuleList(
    (0): LayerNorm((50,), eps=1e-08, elementwise_affine=True)
    (1): LayerNorm((50,), eps=1e-08, elementwise_affine=True)
  )
  (forward_layers): ModuleList(
    (0): PointWiseFeedForward(
      (conv1): Conv1d(50, 50, kernel_size=(1,), stride=(1,))
      (dropout1): Dropout(p=0.3, inplace=False)
      (relu): ReLU()
      (conv2): Conv1d(50, 50, kernel_size=(1

In [14]:
usernum = 0
itemnum = 0
User = defaultdict(list)
user_train = {}
user_valid = {}
user_test = {}
# assume user/item index starting from 1
f = open('data/%s.txt' % 'ml-1m', 'r')
for line in f:
    u, i = line.rstrip().split(',')
    u = int(u)
    i = int(i)
    usernum = max(u, usernum)
    itemnum = max(i, itemnum)
    User[u].append(i)

In [15]:
user_test = defaultdict(list)
for i in tqdm(range(1, usernum+1)):
    j = list(range(1, itemnum+1))
    user_test[i].append(list(set(j).difference(User[i])))
    user_test[i] = user_test[i][0]

100%|██████████████████████████████████████████████████████████████████████████| 53424/53424 [00:46<00:00, 1144.53it/s]


In [ ]:
with open('./sub_f.csv', 'ab') as f:
    maxlen_te = max(len(user_test[i]) for i in range(1, usernum+1))
    maxlen_tr = max(len(User[i]) for i in range(1, usernum+1))
    for i in tqdm(range(1, usernum+1)):
        seq = np.zeros([maxlen_tr], dtype=np.int32)
        idx = maxlen_tr - 1
        for j in reversed(User[i]):
            seq[idx] = j
            idx -= 1
            if idx == -1: break
        item_idx = user_test[i]
        p = [np.array(l) for l in [[i], [seq], item_idx]]
        predictions = -model.predict(*p)
        predictions = predictions[0]
        a = predictions.argsort()[:10]
        a = a.numpy()
        r = np.array(item_idx)
        s = r[a]
        u = np.full(shape=10, fill_value=i, dtype=np.int)
        pre = np.c_[u-1,s-1]
        np.savetxt(f, pre, delimiter=',', fmt='%i')
df = pd.read_csv('./sub_f.csv',header=None,names=['user_id', 'item_id'])
df.to_csv('./submission_1.csv',index=False)

  0%|                                                                                        | 0/53424 [00:00<?, ?it/s]<ipython-input-16-cce695dcabd8>:19: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  u = np.full(shape=10, fill_value=i, dtype=np.int)
  2%|█▎                                                                           | 889/53424 [00:09<07:56, 110.20it/s]